# Programatically Creating OpenFOAM Cases from a Template
In the last document we outline how to convert a STEP file into an OpenFOAM PolyMesh. In this document we outline how to programatically generate OpenFOAM cases using a case as a template. For the current example we want to generate many cases with an inlet diameter specific flow rate using Newtonian boundary conditions. 

## Creating a Base Class
Let's create a base class for reusability. The class takes in some template openfoam case, copies it to some desired directory and then performs some specific modification on case.

In [1]:
import pathlib as pt
import abc
import distutils.dir_util
import numpy as np

class FoamTemplateGenerator(abc.ABC):
    # create a modifiable variable pointing to the location of the template openfoam case
    # for now it will be None
    case_path = None

    def __init__(self, target_path: pt.Path):
        # the target path will be the location where we will be copying our template to
        self.target_path = pt.Path(target_path)

    @abc.abstractmethod
    def modify(self):
        # an abstract modifiable method which allows us to 
        NotImplemented

    def construct(self):
        # construct the case folder from the specified template
        distutils.dir_util.copy_tree(
            self.case_path.absolute().as_posix(),
            self.target_path.absolute().as_posix()
        )
        self.modify()

## Creating a Child Class
We will inherit from the parent class overiding the modify method and creating our own custom initialization. We will specify a inlet diameter specific flow rate.

In [2]:
import pathlib as pt

path_to_template_case = pt.Path("../arterygen/OpenFOAM_templates/newtonian_steady")

class NewtonianSteadyBifurcationGenerator(FoamTemplateGenerator):

    case_path = path_to_template_case

    def __init__(self, diameter, *args):
        # calculate the area
        area = np.pi * (diameter/2)**2
        # create the diameter specific flow rate
        self.inlet_velocity = 1.43*(diameter**2.55)/area
        # not that the first argument will be the target location
        super().__init__(*args)

    def modify(self):
        '''
        The following modify method overrides the existing U file within the
        OpenFoam 0 folder and replaces it with a U file which uses the specified inlet velocity
        '''
        U_file = self.target_path/"0"/"U"
        file_text = [
            "/*--------------------------------*- C++ -*----------------------------------*\\",
            "| =========                 |                                                 |",
            "| \\\\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |",
            "|  \\\\    /   O peration     | Version:  2.1.1                                 |",
            "|   \\\\  /    A nd           | Web:      www.OpenFOAM.org                      |",
            "|    \\\\/     M anipulation  |                                                 |",
            "\\*---------------------------------------------------------------------------*/",
            "FoamFile",
            "{",
            "    version     2.0;",
            "    format      ascii;",
            "    class       volVectorField;",
            "    object      U;",
            "}",
            "// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //",
            "",
            "dimensions      [0 1 -1 0 0 0 0];",
            "",
            "internalField   uniform (0 0 0);",
            "",
            "boundaryField",
            "{",
            "    WALL",
            "    {",
            "        type            fixedValue;",
            "        value           uniform (0 0 0);",
            "    }",
            "",
            "",
            "	INLET",
            "	{",
            "		type     surfaceNormalFixedValue;",
            f"		refValue -{self.inlet_velocity};",
            "	}",
            "",
            "    OUTLET_1",
            "    {",
            "        type            zeroGradient;",
            "    }",
            "",
            "    OUTLET_2",
            "    {",
            "        type            zeroGradient;",
            "    }",
            "}",
            "",
            "// ************************************************************************* //",
        ]
        with open(U_file, "w", newline="\n") as f:
            f.write("\n".join(file_text))

## Creating an Example Case
Now that we have the template generator set up we can generate a sample case

In [3]:
import shutil
inlet_diameter = 0.0036 # diameter is in mm
output_folder = pt.Path("./example_openfoam_case")
# by default the specified folder is not automatically deleted
# we should therefore check whether the folder exists and delete the folder
if output_folder.exists():
    shutil.rmtree(output_folder)
sample_generator = NewtonianSteadyBifurcationGenerator(inlet_diameter, output_folder)
sample_generator.construct()

# Populating the PolyMesh
Now that the OpenFOAM case has been generated we need to populate the PolyMesh folder. To do so we can read in a step file and convert it with the methods from the prior notebook.

In [4]:
# we have created a function using the code from the last notebook
from arterygen.glyph_template import generate_ideal_bifurcation_glyph_template_1

# let us once again create a base class for reusability
class STEPToFoam:
    '''
        Handler class,
        the name of the STEP file will be,
        base_*type*_inlet_*radius*_outlet1_*radius*_outlet2_*radius*,
        converts a step file to OpenFOAM case
    '''

    mesh_files = [
            "boundary", "cellZones", "faces",
            "faceZones", "neighbour", "owner", "points"
    ]

    def __init__(self, target_folder: pt.Path, openfoam_case_constructor):
        target_folder = pt.Path(target_folder)
        # The target folder will be the folder to save the openfoam case to
        self.target_folder = target_folder
        self.openfoam_case_constructor = openfoam_case_constructor

    def __call__(self, case: pt.Path):
        # the case file will be the step file to be converted
        case = pt.Path(case)
        # generate the openfoam case
        self.foam_folder = self.target_folder/case.stem
        if not all(
            [
                (self.foam_folder/"constant/polyMesh"/item).exists()
                    for item in self.mesh_files
            ]
        ):
            self.openfoam_case_constructor(self.foam_folder)
            # generate the PolyMesh
            generate_ideal_bifurcation_glyph_template_1(
                case,
                self.foam_folder/"constant"/"polyMesh",
                dimension_spacing        = 0.2,
                wall_spacing             = 0.025,
                trex_maximum_layers      = 6,
                trex_growth_rate         = 1.1,
                inlet_connector_names     = ("con-1", "con-7"),
                outlet_1_connector_names  = ("con-28", "con-31"),
                outlet_2_connector_names = ("con-35", "con-37")
            )
    def clean(self):
        if self.foam_folder.exists():
            shutil.rmtree(self.foam_folder)

# the following method reads in the name of some folder of the form base_*type*_inlet_*radius*_outlet1_*radius*_outlet2_*radius* 
# and then reads in the diameter. The dimaeter is then passed into the case generator. 
def make_newtonian_steady_case(foam_folder):
    diameter   = float(foam_folder.name.split("_")[3])/1000
    NewtonianSteadyBifurcationGenerator(diameter, foam_folder).construct()
# finally we can package it all in a nicde class to run everything from
class STEPToFoamNewtonianSteadyFoam(STEPToFoam):
    def __init__(self, target_folder: pt.Path):
        super().__init__(target_folder, make_newtonian_steady_case)

In [5]:
# the case will be created in the current folder
step_to_foam_converter = STEPToFoamNewtonianSteadyFoam("./")
step_to_foam_converter("./base_01_inlet_3_bif_1.4_outlet_1.6_angle_0.STEP")

Info: processing base_01_inlet_3_bif_1.4_outlet_1.6_angle_0.STEP
Server: Pointwise V18.3

Info: Units specified in the file: Millimeters (0.001 meters)

Info: Model Size adjusted to 100

Info: Begin: aniso layer   1: growthRate = 1.000000, height = 1.000000, totalHeight = 1.000000

Info: End  : aniso layer   1: numPts =     35125, numTets =    100713, (0.920000 seconds)

Info: Begin: aniso layer   2: growthRate = 1.100000, height = 1.100000, totalHeight = 2.100000

Info: End  : aniso layer   2: numPts =     51914, numTets =    201426, (0.827000 seconds)

Info: Begin: aniso layer   3: growthRate = 1.100000, height = 1.210000, totalHeight = 3.310000

Info: End  : aniso layer   3: numPts =     68691, numTets =    302077, (0.865000 seconds)

Info: Begin: aniso layer   4: growthRate = 1.100000, height = 1.331000, totalHeight = 4.641000

Info: End  : aniso layer   4: numPts =     85357, numTets =    402074, (0.819000 seconds)

Info: Begin: aniso layer   5: growthRate = 1.100000, height = 1.4

In [6]:
# cleanup and delete the generated case
step_to_foam_converter.clean()